In [ ]:
### 20220629 ### STATS: COMMODITIES EXPORT PROPORTION IN OVERALL COUNTRY GOODS & SERVICES EXPORT

gc.collect()
### Stats parameter - leaders to plot:
int_top = 15
### Global export data loading:
df_global_world = pd.read_hdf(path_or_buf = str_path_global, key = str_key_unc, where = 'Flow = Export').droplevel('Flow')
### Proportion calculating:
ser_comm_proportion = (df_comm_world.groupby(['Date', 'Reporter'])['Value'].sum().dropna() * 100 / df_global_world).round(0)
df_comm_proportion = ser_comm_proportion.to_frame().join(ser_ison_status, on = 'Reporter').reset_index()
### Trade leaders countries list:
idx_ave_top = ser_country_ave[: int_top].index
### Other leaders countries list:
idx_ave_other = ser_country_ave[int_top :].index    
### Leaders data extraction for leaders plot:
df_top_part = df_comm_proportion.loc[df_comm_proportion['Reporter'].isin(idx_ave_top)].copy()
df_top_part['Year'] = df_top_part['Date'].dt.year
df_top_part = df_top_part[['Year', 'Reporter', 'Value']].set_index(['Reporter', 'Year']).squeeze().unstack('Year').sort_index()
df_top_part.index = df_top_part.index.astype('O')
### Other countries data extraction for leaders plot:
df_other_part = df_comm_proportion[df_comm_proportion['Reporter'].isin(idx_ave_other)].groupby('Date')['Value'].mean().reset_index()
df_other_part['Year'] = df_other_part['Date'].dt.year
df_other_part = pd.concat({'Other': df_other_part.set_index('Year')}, names = ['Reporter'])
df_other_part = df_other_part['Value'].squeeze().unstack('Year')
### Region's average values:
list_regions_ave = []
for iter_region in list_region:
    df_iter_part = df_comm_proportion[df_comm_proportion['Market'] == iter_region].groupby('Date')['Value'].mean().reset_index()
    df_iter_part['Year'] = df_iter_part['Date'].dt.year
    df_iter_part = pd.concat({iter_region: df_iter_part.set_index('Year')}, names = ['Reporter'])
    df_iter_part = df_iter_part['Value'].squeeze().unstack('Year')
    list_regions_ave.append(df_iter_part)
df_regions = pd.concat(list_regions_ave, axis = 0)
### Leaders plot data aggregation:
df_to_plot = pd.concat([df_top_part, df_other_part, df_regions], axis = 0)
### Leaders plotting:
plt.figure(figsize = (15, 7))
ax_heatmap = sns.heatmap(df_to_plot, cmap = 'YlGnBu', xticklabels = True, yticklabels = True, cbar = True, linewidth = 0.5, square = False, annot = True, 
                         fmt = '0.0f', center = 50, vmin = 0, vmax = 100)
ax_heatmap.set(xlabel = 'Year', ylabel = 'Countries')
ax_heatmap.set_title('top-' + str(int_top) + ': ' + 'Commodities export proportion in total country goods & services export, %' , fontsize = 15)
plt.yticks(rotation = 0)
plt.show()  
### Region's data plotting:
for iter_region in list_region:
    ### Data extraction:    
    df_to_plot = df_comm_proportion.loc[df_comm_proportion['Market'] == iter_region].copy()
    df_to_plot['Year'] = df_to_plot['Date'].dt.year
    df_to_plot = df_to_plot[['Year', 'Reporter', 'Value']].set_index(['Reporter', 'Year']).squeeze().unstack('Year').sort_index()
    ### Results plotting:
    plt.figure(figsize = (15, 7))
    ax_heatmap = sns.heatmap(df_to_plot, cmap = 'YlGnBu', xticklabels = True, yticklabels = True, cbar = True, linewidth = 0.5, square = False, annot = True, 
                             fmt = '0.0f', center = 50, vmin = 0, vmax = 100)
    ax_heatmap.set(xlabel = 'Year', ylabel = 'Countries')
    ax_heatmap.set_title(iter_region + ': ' + ' Proportion of commodities export in total country goods & services export, %' , fontsize = 15)
    plt.yticks(rotation = 0)
    plt.show()  

In [ ]:
### IMPORT OBSERVATIONS INCORPORATION RESULTS ILLUSTRATING: BEFORE COMLPLETING

gc.collect
### Stats parameter:
flo_error = -0.01
### Top number:
int_top = 15
### ISON total trade:
ser_comm_ison = df_comm_full.groupby(['Date', 'Reporter', 'Commodity_Name'])['Value'].sum()#.dropna()
ser_comm_ison.name = 'Total_ISON'
### Difference caclulating:
ser_comm_world = df_comm_world.set_index(['Date', 'Reporter', 'Commodity_Name'])['Value']
ser_comm_world.name = 'World'
df_comm_sum = pd.concat([ser_comm_world, ser_comm_ison], axis = 1).dropna() / 1000000000
df_comm_sum = df_comm_sum.groupby(['Date', 'Reporter']).sum()
ser_comm_diff = (1 - df_comm_sum['Total_ISON'] / df_comm_sum['World']) * 100
ser_comm_diff.name = 'Not_Covered'
ser_comm_value = (df_comm_sum['World'] - df_comm_sum['Total_ISON']).clip(lower = 0).astype(int)
ser_comm_value.name = 'Not_Covered'
print('ISON sum > World sum:')
display(ser_comm_diff[ser_comm_diff < flo_error])
ser_comm_ave = ser_comm_diff.groupby('Reporter').mean().sort_values(ascending = False)
### Externat trade countries list:
idx_ave_top = ser_comm_ave[: int_top].index
### Other countries list:
idx_ave_other = ser_comm_ave[int_top :].index  
### Leaders data extraction for leaders plot:
df_top_part = ser_comm_diff.loc[All, idx_ave_top].reset_index()
df_top_part['Year'] = df_top_part['Date'].dt.year
df_top_part = df_top_part[['Year', 'Reporter', 'Not_Covered']].set_index(['Reporter', 'Year']).squeeze().unstack('Year').sort_index()
df_top_part.index = df_top_part.index.astype('O')
### Other countries data extraction for leaders plot:
df_other_part = ser_comm_diff.loc[All, idx_ave_other].reset_index().groupby('Date')['Not_Covered'].mean().reset_index()
df_other_part['Year'] = df_other_part['Date'].dt.year
df_other_part = pd.concat({'Other': df_other_part.set_index('Year')}, names = ['Reporter'])
df_other_part = df_other_part['Not_Covered'].squeeze().unstack('Year')
### Leaders plot data aggregation:
df_to_plot = pd.concat([df_top_part, df_other_part], axis = 0)
### Leaders data extraction for leaders plot:
df_top_part = ser_comm_value.loc[All, idx_ave_top].reset_index()
df_top_part['Year'] = df_top_part['Date'].dt.year
df_top_part = df_top_part[['Year', 'Reporter', 'Not_Covered']].set_index(['Reporter', 'Year']).squeeze().unstack('Year').sort_index()
df_top_part.index = df_top_part.index.astype('O')
### Other countries data extraction for leaders plot:
df_other_part = ser_comm_value.loc[All, idx_ave_other].reset_index().groupby('Date')['Not_Covered'].sum().reset_index()
df_other_part['Year'] = df_other_part['Date'].dt.year
df_other_part = pd.concat({'Other': df_other_part.set_index('Year')}, names = ['Reporter'])
df_other_part = df_other_part['Not_Covered'].squeeze().unstack('Year')
### Leaders plot data aggregation:
df_to_anotate = pd.concat([df_top_part, df_other_part], axis = 0)
### Leaders plotting:
plt.figure(figsize = (15, 7))
ax_heatmap = sns.heatmap(df_to_plot, cmap = 'YlGnBu', xticklabels = True, yticklabels = True, cbar = True, linewidth = 0.5, square = False, annot = df_to_anotate, 
                         fmt = '0.0f', center = 50, vmin = 0, vmax = 100)
ax_heatmap.set(xlabel = 'Year', ylabel = 'Countries')
ax_heatmap.set_title('Top-' + str(int_top) + ' Out-of-ISON Exporters, billions of USD\n' + 
                     'BEFORE Integration of Import Observations:\n' + 
                     '(color bar - Proportion of Out-of-ISON Export to Total Export, %)' , fontsize = 15)
plt.yticks(rotation = 0)
plt.show()  

In [ ]:
### IMPORT OBSERVATIONS INCORPORATION RESULTS ILLUSTRATING: AFTER COMLPLETING

gc.collect
### Stats parameter:
flo_error = -0.01
### Top number:
int_top = 15
### Import bilateral data loading and inverting:
df_import = get_comtrade_data(str_flow = 'Import', str_flag = [0, 1]).reset_index()
df_import_inverted = df_import[['Date', 'Partner', 'Reporter', 'Commodity_Name', 'Value']]
df_import_inverted.columns = ['Date', 'Reporter', 'Partner', 'Commodity_Name', 'Value']
ser_import_inverted = df_import_inverted.set_index(['Date', 'Reporter', 'Partner', 'Commodity_Name'])['Value']
ser_import_inverted.name = 'Import_Inverted'
### Export preparing:
ser_export = df_comm_full.set_index(['Date', 'Reporter', 'Partner', 'Commodity_Name'])['Value'].astype('int64')
ser_export.name = 'Export'
df_comm_augmented = ser_export.combine_first(ser_import_inverted).reset_index()
df_comm_augmented.rename({'Export': 'Value'}, axis = 1, inplace = True)
### ISON total trade:
ser_comm_ison = df_comm_augmented.groupby(['Date', 'Reporter', 'Commodity_Name'])['Value'].sum()#.dropna()
ser_comm_ison.name = 'Total_ISON'
### Difference caclulating:
ser_comm_world = df_comm_world.set_index(['Date', 'Reporter', 'Commodity_Name'])['Value']
ser_comm_world.name = 'World'
df_comm_sum = pd.concat([ser_comm_world, ser_comm_ison], axis = 1).dropna() / 1000000000
df_comm_sum = df_comm_sum.groupby(['Date', 'Reporter']).sum()
ser_comm_diff = (1 - df_comm_sum['Total_ISON'] / df_comm_sum['World']) * 100
ser_comm_diff.name = 'Not_Covered'
ser_comm_value = (df_comm_sum['World'] - df_comm_sum['Total_ISON']).clip(lower = 0).astype(int)
ser_comm_value.name = 'Not_Covered'
### Leaders data extraction for leaders plot:
df_top_part = ser_comm_diff.loc[All, idx_ave_top].reset_index()
df_top_part['Year'] = df_top_part['Date'].dt.year
df_top_part = df_top_part[['Year', 'Reporter', 'Not_Covered']].set_index(['Reporter', 'Year']).squeeze().unstack('Year').sort_index()
df_top_part.index = df_top_part.index.astype('O')
### Other countries data extraction for leaders plot:
df_other_part = ser_comm_diff.loc[All, idx_ave_other].reset_index().groupby('Date')['Not_Covered'].mean().reset_index()
df_other_part['Year'] = df_other_part['Date'].dt.year
df_other_part = pd.concat({'Other': df_other_part.set_index('Year')}, names = ['Reporter'])
df_other_part = df_other_part['Not_Covered'].squeeze().unstack('Year')
### Leaders plot data aggregation:
df_to_plot = pd.concat([df_top_part, df_other_part], axis = 0)
### Leaders data extraction for leaders plot:
df_top_part = ser_comm_value.loc[All, idx_ave_top].reset_index()
df_top_part['Year'] = df_top_part['Date'].dt.year
df_top_part = df_top_part[['Year', 'Reporter', 'Not_Covered']].set_index(['Reporter', 'Year']).squeeze().unstack('Year').sort_index()
df_top_part.index = df_top_part.index.astype('O')
### Other countries data extraction for leaders plot:
df_other_part = ser_comm_value.loc[All, idx_ave_other].reset_index().groupby('Date')['Not_Covered'].sum().reset_index()
df_other_part['Year'] = df_other_part['Date'].dt.year
df_other_part = pd.concat({'Other': df_other_part.set_index('Year')}, names = ['Reporter'])
df_other_part = df_other_part['Not_Covered'].squeeze().unstack('Year')
### Leaders plot data aggregation:
df_to_anotate = pd.concat([df_top_part, df_other_part], axis = 0)
### Leaders plotting:
plt.figure(figsize = (15, 7))
ax_heatmap = sns.heatmap(df_to_plot, cmap = 'YlGnBu', xticklabels = True, yticklabels = True, cbar = True, linewidth = 0.5, square = False, annot = df_to_anotate, 
                         fmt = '0.0f', center = 50, vmin = 0, vmax = 100)
ax_heatmap.set(xlabel = 'Year', ylabel = 'Countries')
ax_heatmap.set_title('Top-' + str(int_top) + ' Out-of-ISON Exporters, billions of USD\n' + 
                     'AFTER Integration of Import Observations:\n' + 
                     '(color bar - Proportion of Out-of-ISON Export to Total Export, %)' , fontsize = 15)
plt.yticks(rotation = 0)
plt.show()  